### Remove DB

In [1]:
import os

if os.path.exists('example.db'):
    os.remove('example.db')

### Setup DB

In [2]:
from sqlalchemy import create_engine
from sqlalchemy.orm import Session

from param_persist.sqlalchemy.models import Base


engine = create_engine('sqlite:///example.db')
Base.metadata.create_all(engine)

connection = engine.connect()
transaction = connection.begin()
session = Session(bind=connection)

### Param Class

In [3]:
import param
import numpy as np

class ExampleParamClass(param.Parameterized):
    number_field = param.Number(0.5, doc="A simple number field.")
    integer_field = param.Integer(1, doc="A simple integer field.")
    string_field = param.String("My String", doc="A simple string field.")
    bool_field = param.Boolean(False, doc="A simple boolean field.")
    

### Create Agent

In [4]:
from param_persist.agents.sqlalchemy_agent import SqlAlchemyAgent
from param_persist.sqlalchemy.models import InstanceModel, ParamModel

agent = SqlAlchemyAgent(engine=engine)

In [5]:
def show_database(db_session):
    print('Instances:')
    for parameterized_instance in db_session.query(InstanceModel).all():
        print(f'\t{parameterized_instance}')
    
    print('Parameters:')
    for parameter in db_session.query(ParamModel).all():
        print(f'\t{parameter}\n\t\t{parameter.value}')
    

### Save a Parameterized Object

In [6]:
show_database(session)

my_param = ExampleParamClass()
my_param_id = agent.save(my_param)

show_database(session)


Instances:
Parameters:
Instances:
	<Instance(id="6a5a89a4-9268-4867-b6c7-3ae641f0c963", class_path="__main__.ExampleParamClass")>
Parameters:
	<Param(id="fbc03b12-a493-482a-9ae5-246b57afbab6", instance_id="6a5a89a4-9268-4867-b6c7-3ae641f0c963")>
		{"name": "number_field", "value": 0.5, "type": "param.Number"}
	<Param(id="aecd7a9d-116e-4f74-aa6d-61676dc2570b", instance_id="6a5a89a4-9268-4867-b6c7-3ae641f0c963")>
		{"name": "integer_field", "value": 1, "type": "param.Integer"}
	<Param(id="7b3de963-675c-40fe-8199-6b2bfc0750c4", instance_id="6a5a89a4-9268-4867-b6c7-3ae641f0c963")>
		{"name": "string_field", "value": "My String", "type": "param.parameterized.String"}
	<Param(id="3cf98040-095b-4ced-90e0-5485f7ff1bb3", instance_id="6a5a89a4-9268-4867-b6c7-3ae641f0c963")>
		{"name": "bool_field", "value": false, "type": "param.Boolean"}



### Update a Parameterized Object

In [7]:
my_param.number_field = 2.2
my_param.bool_field = True

agent.update(my_param, my_param_id)

show_database(session)

Instances:
	<Instance(id="6a5a89a4-9268-4867-b6c7-3ae641f0c963", class_path="__main__.ExampleParamClass")>
Parameters:
	<Param(id="fbc03b12-a493-482a-9ae5-246b57afbab6", instance_id="6a5a89a4-9268-4867-b6c7-3ae641f0c963")>
		{"name": "number_field", "value": 2.2, "type": "param.Number"}
	<Param(id="aecd7a9d-116e-4f74-aa6d-61676dc2570b", instance_id="6a5a89a4-9268-4867-b6c7-3ae641f0c963")>
		{"name": "integer_field", "value": 1, "type": "param.Integer"}
	<Param(id="7b3de963-675c-40fe-8199-6b2bfc0750c4", instance_id="6a5a89a4-9268-4867-b6c7-3ae641f0c963")>
		{"name": "string_field", "value": "My String", "type": "param.parameterized.String"}
	<Param(id="3cf98040-095b-4ced-90e0-5485f7ff1bb3", instance_id="6a5a89a4-9268-4867-b6c7-3ae641f0c963")>
		{"name": "bool_field", "value": true, "type": "param.Boolean"}


### Load Parameterized Object

In [8]:
my_param_loaded = agent.load(my_param_id)

print(my_param_loaded.bool_field)
print(my_param_loaded.number_field)
print(my_param_loaded.integer_field)
print(my_param_loaded.string_field)

True
2.2
1
My String


### Delete Parameterized Object

In [9]:
show_database(session)
agent.delete(my_param_id)
show_database(session)

Instances:
	<Instance(id="6a5a89a4-9268-4867-b6c7-3ae641f0c963", class_path="__main__.ExampleParamClass")>
Parameters:
	<Param(id="fbc03b12-a493-482a-9ae5-246b57afbab6", instance_id="6a5a89a4-9268-4867-b6c7-3ae641f0c963")>
		{"name": "number_field", "value": 2.2, "type": "param.Number"}
	<Param(id="aecd7a9d-116e-4f74-aa6d-61676dc2570b", instance_id="6a5a89a4-9268-4867-b6c7-3ae641f0c963")>
		{"name": "integer_field", "value": 1, "type": "param.Integer"}
	<Param(id="7b3de963-675c-40fe-8199-6b2bfc0750c4", instance_id="6a5a89a4-9268-4867-b6c7-3ae641f0c963")>
		{"name": "string_field", "value": "My String", "type": "param.parameterized.String"}
	<Param(id="3cf98040-095b-4ced-90e0-5485f7ff1bb3", instance_id="6a5a89a4-9268-4867-b6c7-3ae641f0c963")>
		{"name": "bool_field", "value": true, "type": "param.Boolean"}
Instances:
Parameters:
